### Parse Bank Statements
--This program reads in a text file containing information from one or more bank statements and returns a dataframe with all transaction information, including: date, transaction category,  description, and amount. It also consolidates "marginal" information, e.g. monthly begining and ending balance, and performs automated accuracy checks.  




Assumptions: 
-- Amount value can be positive, or negative, less than a doller, including $0.00
--In wrappred trxns, i.e. those comprised of more than one paragraph ("graph"), the last graphs contains only amount info

##### Import packages 

In [1]:
import pandas as pd
import numpy as np
import os
import re
from datetime import datetime
import docx   # Python library for creating, updating, also reading MS Word .docx files

##### Functions to load and clean data, and identify location of monthly statements

In [2]:
###   Read in Microsoft Word .docx file 
def get_doc(path):
    
    doc_obj = docx.Document(path)
    
    return doc_obj

###  Clean text
def clean_text(text):
    
    text1 = ''.join([x for x in text if ord(x) < 128])    #Remove non-ascii unicode
    text2 = text1.lower()    #Make lowercase
    text3 = text2.replace(' ', '')    #Strip out whitespace 
    
    return text3 

###  Split document into paragrpahs ("graphs"), clean each graph, then append to a list 
def get_graph_text(document):
    
    original_graphs = []
    clean_graphs = []
    
    for paragraph in document.paragraphs:        
        original_graphs.append(paragraph.text)
        
    for i in range(len(original_graphs)):
        clean_graphs.append(clean_text(original_graphs[i]))
                                
    return original_graphs, clean_graphs

##### Functions to support conforming transactions and creating dataframes 

In [3]:
###   Find locations of compiled regex patterns, return as a list 
def get_pattern_loc(pattern, graph_list):
    
    pattern_loc = []
    pattern_graphs = []
    
    for i in range(len(graph_list)):
    
        if bool(re.search(pattern, graph_list[i])):
        
            pattern_loc.append(i)
            pattern_graphs.append(graph_list[i])
        
        else:
            continue
            
    return pattern_loc, pattern_graphs   

In [4]:
def get_trxn_year(date_loc_list, date_graph_list):

    stmnt_year_list =[]

    for i in range(len(date_loc_list)):

        graph_year = re.search('\d{4}\Z', date_graph_list[i]).group()
        stmnt_year_list.append(graph_year)

    year_set = set(stmnt_year_list)
    unique_years = list(year_set)

    if len(unique_years) == 1:
        trxn_year = unique_years[0]

    else:
        print('ERROR: Statment dates contain mutiple years')
        trxn_year = '9999'    
    
    return trxn_year

In [5]:
###   Set boolean variable to indicate a postive or negative amount, based on transaction type
def check_add_bal(trxn_type):
    
    postive_change =set(['Deposit', 'ACHcredit', 'OtherCredit'])
    
    if trxn_type in postive_change:
        add_balance = True 
        
    else:
        add_balance = False 
        
    return add_balance       

In [6]:
###   Convert transaction amount text to float number 
def get_amount(amt_text):
    
    neg_amt_flag = False
    
    amt_text = amt_text.replace(',','')
    amt_text = amt_text.replace('$','')
    
    if bool(re.match(r'-', amt_text)):
        neg_amt_flag = True
            
    amt_text = amt_text.replace('-','')
    
    fraction_span = re.search(r'\.\d{2}', amt_text).span()
    
    fraction = int(amt_text[fraction_span[0]+1:])/100
    
    if fraction_span[0] == 0:
        integer = 0
        
    else:
        integer = int(amt_text[:fraction_span[0]])
    
    amount_num = integer + fraction 
    
    if neg_amt_flag:
        amount_num = amount_num *-1
    
    return amount_num   

In [7]:
###   Insert dollar sign, to simplify converting amount from text to float 
def insert_USDsign(trxn, USD_type):

    if USD_type == 'Deposit': 
        
        text_sub =re.compile('deposit')
        new_trxn = text_sub.sub('deposit$', trxn)        
        
    elif USD_type == 'ACHcredit' or USD_type =='ACHdebit':
        
        date_start = re.match('\d{2}-\d{2}', trxn).start()
        date_end = re.match('\d{2}-\d{2}', trxn).end()
        date = trxn[date_start : date_end]
        date = date.replace('-', '')
        amt_bdry = re.compile(date)        
        new_trxn = amt_bdry.sub(date +'$', trxn)            
        
    elif USD_type == 'ATMcardPOS':
        
        sub_group = re.search(r'x{12}\d{4}',trxn).group()        
        sub_pattern = re.compile(sub_group)
        new_trxn = sub_pattern.sub(sub_group+'$', trxn)
        
    elif USD_type == 'OtherCredit' or USD_type == 'OtherDebit':
        
        if bool(re.search(r'overdraftpospurchase', trxn)):
            
            new_trxn = trxn.replace(',','')
            descrip_text_end = re.search(r'overdraftpospurchase', new_trxn).end()
            sub_group = new_trxn[:descrip_text_end+15]
            sub_pattern = re.compile(sub_group)
            new_trxn = sub_pattern.sub(sub_group+'$', new_trxn)
            
        else:          
            
            new_trxn = trxn.replace(',','')
            amount_start= re.search(r'\d+?\.\d{2}\Z', new_trxn).start()
            sub_group = new_trxn[:amount_start]
            sub_pattern = re.compile(sub_group)
            new_trxn = sub_pattern.sub(sub_group + '$', new_trxn)      
        
    else:
        new_trxn = trxn   
        
    return new_trxn

In [8]:
###  Replace "clean" transaction description with text from original document
def get_description(locate, amount_txt):
   
    if type(locate) is list:

        new_descript = ''
        
        for i in range(len(locate)-1):            
            new_descript = new_descript + original_graphs[locate[i]]
            
            
        date_end = re.match(r'\d{2}-\d{2}', new_descript).end()    
        new_descript = new_descript[date_end:]
 
    else: 
        
        original_trxn = original_graphs[locate]
        date_end = re.match(r'\d{2}-\d{2}', original_trxn).end()
        
        amount = amount_txt.replace('$','')
        amount = amount.replace('-','')        
        amount_search_pttn = re.compile(r'({})\Z'.format(amount))     
        amount_start = re.search(amount_search_pttn, original_trxn).start()
        
        new_descript = original_trxn[date_end:amount_start]
        
    new_descript = new_descript.lstrip()
    
    return new_descript

In [9]:
###   Reassemble, or "unwrap" trxns contained in multiple graphs, 
def unwrap_trxn_graphs(loc_list, trxn_graph_list):
    
    unsplit_graphs=[]
    
    for loc in loc_list:
        
        if not bool(re.search('\.\d{2}\Z', trxn_graph_list[loc])):            
             unsplit_graphs.append(trxn_graph_list[loc])
                
        else:
                       
            if not bool(re.match('$', trxn_graph_list[loc])):
                
                temp = '$'+trxn_graph_list[loc]
                unsplit_graphs.append(temp)               
                                 
            else:
                unsplit_graphs.append(trxn_graph_list[loc])
                
    unsplit_trxn = ''.join(unsplit_graphs)
    
    return unsplit_trxn

##### Functions to conform transactions and create data frames 

In [10]:
###   Create dataframe containing transaction date, description,from location of transaction graphs 
def get_trxn_df(trxn_list, trxn_type, descript_loc):
    
    date_txt_list = []
    description_list = []
    amount_list = []

    add_bal = check_add_bal(trxn_type)

    for i in range(len(trxn_list)):
        
        trxn = trxn_list[i]
        date_txt = trxn_year + '-' + re.search(r'\d{2}-\d{2}', trxn).group() 
        date_end = re.search(r'\d{2}-\d{2}', trxn).end() 

        amount = get_amount(re.search(amount_pattern, trxn).group())
        amount_begin = re.search(amount_pattern, trxn).start()
        amount_txt = trxn[amount_begin:]
        
        if trxn_type == 'Check':
            description = trxn[date_end: amount_begin]
            
        else:
            description = get_description(descript_loc[i], amount_txt) 
        
        date_txt_list.append(date_txt)
        description_list.append(description)
        amount_list.append(amount)

    trxn_lists = list(zip(date_txt_list, description_list, amount_list))

    trxn_df = pd.DataFrame(trxn_lists, columns=['Trxn_Date', 'Description', 'Amount'])
    
    for t in range(len(trxn_df)):
        trxn_df.loc[t, 'Trxn_Date'] = datetime.strptime(trxn_df.loc[t, 'Trxn_Date'], "%Y-%m-%d")
   
    trxn_df['Trxn Type'] = friendly_trxn_name_dict[trxn_type]
    
    if not add_bal:        
        trxn_df['Amount'] = trxn_df.Amount * -1

    return trxn_df 

In [11]:
### Conform check transaction graphs to standard format for dataframe creation
def conform_check_trxn(loc_list, graph_list, amount_pattern):
    
    check_list=[]
    
    for loc in loc_list:
        
        trxn_graph = graph_list[loc]
        
        out_of_seq = True if bool(re.match(r'\*', trxn_graph)) else False
        
        trxn_graph = trxn_graph.replace('*','')
        
        check_num = re.match(r'\d{4}', trxn_graph).group()
        check_amount = re.search(amount_pattern, trxn_graph).group()
        check_date = re.search(r'\d{2}-\d{2}\Z', trxn_graph).group()
        
        if out_of_seq:
            
            description = 'Check#:' + check_num + '(Out of sequence)'
            
        else: 
            
            description = 'Check#:' + check_num 
            
        new_trxn = check_date + description + check_amount        
        check_list.append(new_trxn)
        
    return check_list 

In [12]:
###   Conform non-check transactions to standard format for dataframe creation
def conform_trxn_list (loc_list, graph_list, amount_pattern, trxn_type):

    conform_trxn = []
    descript_loc = []
   
    if trxn_type == 'Check':        
        conform_trxn = conform_check_trxn(loc_list, graph_list, amount_pattern)
        
    else: 

        for i in range(len(loc_list)):

            if bool(re.search('\.\d{2}\Z', graph_list[loc_list[i]])): # Test to identify non-wrapped transactions 

                conform_trxn.append(insert_USDsign(graph_list[loc_list[i]], trxn_type))     
                
                descript_loc.append(loc_list[i])   #Captures location (in all_graph_text) of trxn
                
            else: 

                wrapped_graph_loc=[]

                t=loc_list[i]

                while not bool(re.search('\.\d{2}\Z', graph_list[t])):

                    wrapped_graph_loc.append(t)
                    t= t+1
                    
                wrapped_graph_loc.append(t)

                conform_trxn.append(unwrap_trxn_graphs(wrapped_graph_loc, graph_list))
                
                descript_loc.append(wrapped_graph_loc)   #Captures list of wrapped trxn components 

    conform_df = get_trxn_df(conform_trxn, trxn_type, descript_loc)
    
    return conform_df

In [13]:
###   Conform marginal information, to standard format for dataframe creation 
def conform_margin_info(info_list, info_type):
    
    conformed_list=[]
    
    for info in info_list:

        if info_type == 'StmntDate':
            
            non_date_end = re.match('\S+\:', info).end()
            date_text = info[non_date_end:]            
            date = datetime.strptime(date_text, '%B%d,%Y')            
            stmnt_date = date.strftime('%Y-%m-%d')
            
            conformed_list.append(date)            
            
        elif info_type == 'AcctNum':
            
            acct_num = re.search('\d+\Z', info).group()
            conformed_list.append(acct_num)
            
        else:
            
            amount = re.search(amount_pattern, info).group()
            conform_amt = get_amount(amount)
            conformed_list.append(conform_amt)
            
    conformed_df = pd.DataFrame(conformed_list, columns = [info_type])
    
    if info_type == 'WdrawalSubs':
        conformed_df[info_type] = conformed_df[info_type] *-1
    
    return conformed_df

## Main Program

##### Compile regex patterns and define related dictionaries 

In [14]:
### Define regular expression search patterns 

# Patterns for transactions 
amount_pattern = re.compile(r'(((\-?\$\d+|d{1,3})(,\d{3})+)|(\-?\$\d{0,3}))(\.\d{2})')

ACHdebit_pattern = re.compile(r'\d{2}-\d{2}(achdebit)') #'\S+[.]d{2}')

ACHcredit_pattern = re.compile(r'\d{2}-\d{2}(achcredit)') #'\S+[.]d{2}')

ATM_Card_POS_pattern = re.compile(r'(pospurchaseterminal)|(cashwithdrawalterminal)')

Check_pattern =  re.compile(r'(\*?\d{4})(((\$\d+|d{1,3})(,\d{3})+)|(\$\d{0,3}))(\.\d{2})(\d{2}-\d{2})')

Deposit_pattern = re.compile(r'\d{2}-\d{2}(deposit)(((\d+|d{1,3})(,\d{3})+)|(\d{1,3}))(\.\d{2})')

OtherCredit_pattern = re.compile(r'(fundstransferfrmdep)|(wiretransfer-?in)|(atmdeposit)')

OtherDebit_pattern = re.compile(r'(wiretransfer-?out)|(servicecharge)|(otcwithdrawal)|(pospurchasemerchant)|(overdraftfeeforoverdraft)')


# Patterns for marginal information
page1_pattern = re.compile(r'(page:1)')

statement_date_pattern = re.compile('r(statement)')

statement_date_pattern = re.compile(r'(thisstatement)')

acct_number_pattern = re.compile(r'(accountnumber)\d{4,12}')

low_bal_pattern = re.compile(r'(lowbalance)')

avg_bal_pattern = re.compile(r'(averagebalance)')

interest_earned_pattern = re.compile(r'(interestearned)')

begin_bal_pattern = re.compile(r'(beginningbalance)')               

deposits_adds_pattern = re.compile(r'\A(deposits/additions)')

withdrawal_subs_pattern = re.compile(r'\A(withdrawals/subtractions)')

ending_bal_pattern = re.compile(r'(endingbalance)')


In [15]:
### Create dictionaries for compiled search patterns 

# Dictionary for consolidating transaction information 
compiled_pattern_list = [ACHcredit_pattern, ACHdebit_pattern, ATM_Card_POS_pattern, Check_pattern, Deposit_pattern, OtherCredit_pattern, OtherDebit_pattern]
Trxn_type_list = ['ACHcredit', 'ACHdebit', 'ATMcardPOS', 'Check', 'Deposit', 'OtherCredit', 'OtherDebit']
Trxn_type_dict = dict(zip(Trxn_type_list, compiled_pattern_list))

friendly_trxn_type_list = ['ACH Credit', 'ACH Debit', 'Card: ATM/POS', 'Check', 'Deposit', 'Other Credit', 'Other Debit']
friendly_trxn_name_dict = dict(zip(Trxn_type_list, friendly_trxn_type_list))

# Dictionary for consolidating marginal information 
margin_info_patterns = [statement_date_pattern, acct_number_pattern, low_bal_pattern, avg_bal_pattern, interest_earned_pattern, begin_bal_pattern, deposits_adds_pattern, withdrawal_subs_pattern,  ending_bal_pattern]
margin_info_type_list = ['StmntDate', 'AcctNum', 'LowBal', 'AvgBal', 'IntEarned', 'BeginBal', 'DepositsAdds', 'WdrawalSubs', 'EndBal']
margin_info_pattern_dict = dict(zip(margin_info_type_list, margin_info_patterns))

##### Define file paths 

In [16]:
###   Define document locations 

pdf_path = 'C:\\Users\\grego\\OneDrive\\...
text_path = 'C:\\Users\\grego\\OneDrive\\...

##### Main Program

In [17]:
###   Read in word document and create list of clean text paragraphs 

bank_doc = get_doc(text_path)

original_graphs, all_graph_text = get_graph_text(bank_doc)

# Get statment date
stmnt_date_loc, stmnt_date_graphs = get_pattern_loc(statement_date_pattern, all_graph_text)
trxn_year = get_trxn_year(stmnt_date_loc, stmnt_date_graphs)

trxn_year

# Use trxn search patterns to locate trxns; next conform into standard format, then create dataframe 
type_df_list = []    

for T_type in Trxn_type_list:

    pattern_type = Trxn_type_dict[T_type]
    pattern_loc, pattern_trxn = get_pattern_loc(pattern_type, all_graph_text)
    pattern_df = conform_trxn_list(pattern_loc, all_graph_text, amount_pattern, T_type)
    
    type_df_list.append(pattern_df)
    
bank_trxn_df = pd.concat(type_df_list).reset_index(drop=True)


##### Find bank statement marginal information
# Use trxn search patterns to locate marginal info; next conform into standard format, then create dataframe 
info_df_list=[]

for info_type in margin_info_type_list:
    
    pattern_type = margin_info_pattern_dict[info_type]
    pattern_loc, pattern_trxn = get_pattern_loc(pattern_type, all_graph_text)
    info_df_list.append(conform_margin_info(pattern_trxn, info_type))

margin_info_df = pd.concat(info_df_list, axis=1, sort=False)

# Create a new column in %Y-%m format and set as index for margin info
for i in range(len(margin_info_df)):
        margin_info_df.loc[i,'MonthIndex'] = margin_info_df.loc[i,'StmntDate'].strftime('%Y-%m')

margin_info_df.set_index('MonthIndex', inplace = True)

##### Check transaction dataframe accuracy using marginal info

In [19]:
# Create a Month Index in the bank trxn dataframe 
for i in range(len(bank_trxn_df)):
        bank_trxn_df.loc[i,'MonthIndex'] = bank_trxn_df.loc[i,'Trxn_Date'].strftime('%Y-%m')

# Caculate monthly transaction totals, based on the transaction dataframe
monthly_total = bank_trxn_df.loc[:,['MonthIndex', 'Amount']].groupby(['MonthIndex']).agg('sum')

# Create lists for consolidating test results 
months = []
results = []
difference =[]


# Compare montly transaction totals to marginal information 
for i in range(len(monthly_total)):
    
    month_idx = monthly_total.index[i]
    months.append(month_idx)

    deposits_adds= margin_info_df.loc[month_idx, 'DepositsAdds']
    withdrawal_subs = margin_info_df.loc[month_idx, 'WdrawalSubs']  
    info_trxn_total = deposits_adds + withdrawal_subs
    
    diff = info_trxn_total - monthly_total.loc[month_idx,'Amount']
    difference.append(diff)
    
    # Test to see if the results are accurate 
    if monthly_total.loc[month_idx,'Amount'] == info_trxn_total:
        test = 'Pass'
        
    elif diff <= 0.0e-5:
        test = 'Pass'
        
    else:
        test = 'Fail'
    
    results.append(test)
    
    test_lists = list(zip(months, results, difference))
    
    testresults_df = pd.DataFrame(test_lists, columns=['Month', 'Trxn Sum Test', 'Difference'])
        
testresults_df

,Month,Trxn Sum Test,Difference
0,2020-04,Pass,0.000000e+00
1,2020-05,Pass,0.000000e+00
2,2020-06,Pass,0.000000e+00
3,2020-07,Pass,-3.637979e-12
4,2020-08,Pass,-1.818989e-12
5,2020-09,Pass,0.000000e+00
6,2020-10,Pass,0.000000e+00
7,2020-11,Pass,0.000000e+00


##### Write dataframes to Excel file

In [20]:
with pd.ExcelWriter(r'C:\\Users\\grego\\OneDrive\\...,
                   date_format='%Y-%m-%d', datetime_format='YYYY-MM-DD') as bank_trxn_writer:
    
    bank_trxn_df.to_excel(bank_trxn_writer, sheet_name='2020_Nov_Apr_x0257', index=False)
    margin_info_df.to_excel(bank_trxn_writer, sheet_name='Margin Info', index=False)
    testresults_df.to_excel(bank_trxn_writer, sheet_name='Test Results', index=False)
    